# Rainfall Prediction Project

## Data Loading, combining and EDA

*Group 12*

------------

In [3]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
import rpy2.rinterface
import altair as alt

## Download the data

In [3]:
article_id = 14096681
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "../data/raw/figsharerainfall/"

In [4]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)
files = data["files"]

In [5]:
files_to_dl = ["data.zip"]
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

In [6]:
with zipfile.ZipFile(os.path.join(output_directory, files_to_dl[0]), 'r') as f:
    f.extractall(output_directory)

## Combine the data

In [7]:
%%time
exclude = "observed_daily_rainfall_SYD.csv"
files = glob.glob('../data/raw/figsharerainfall/*.csv')
df = pd.concat(
    (pd.read_csv(file, index_col=0)
                .assign(model=re.findall(r'[A-Z][^_]+', file)[0])
                for file in files if file is not exclude)
)
df.to_csv("../data/processed/combined_data.csv")

CPU times: user 6min 1s, sys: 8.78 s, total: 6min 9s
Wall time: 6min 12s


In [8]:
df = pd.read_csv("../data/processed/combined_data.csv")
df.head()

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.244226e-13,MPI-ESM-1-2-HAM
1,1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.217326e-13,MPI-ESM-1-2-HAM
2,1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.498125e-13,MPI-ESM-1-2-HAM
3,1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.251282e-13,MPI-ESM-1-2-HAM
4,1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.270161e-13,MPI-ESM-1-2-HAM


In [9]:
df.tail()

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
62513858,2014-12-27 12:00:00,-30.157068,-29.21466,153.125,154.375,6.689683,SAM0-UNICON
62513859,2014-12-28 12:00:00,-30.157068,-29.21466,153.125,154.375,7.862555,SAM0-UNICON
62513860,2014-12-29 12:00:00,-30.157068,-29.21466,153.125,154.375,10.005026,SAM0-UNICON
62513861,2014-12-30 12:00:00,-30.157068,-29.21466,153.125,154.375,8.541592,SAM0-UNICON
62513862,2014-12-31 12:00:00,-30.157068,-29.21466,153.125,154.375,68.117489,SAM0-UNICON


## Combine data csv on different machines

- Compare observations:

| Team Member   | Operating System | RAM | Processor | Is SSD | Time taken |
|:-------------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Vera Cui      | macOS            | 16GB| M1        | No     | 6min 39s   |
| Lynn Wu       | macOS            | 8GB | M1        | Yes    |  6min 7s   |
| Jasmine Ortega|  macOS         |  8GB |   M1   |  Yes   | 9min 56s     |
| Maeve Shi   | MacOS Big Sur    | 8GB | 2.3 GHz Dual-Core Intel Core i5 | Yes |  7min 30s   |

--------------

##  Load csv and perform EDA on different machines

#### Baseline `read_csv` time

In [10]:
%%time
df = pd.read_csv("../data/processed/combined_data.csv")

CPU times: user 42.7 s, sys: 8.8 s, total: 51.5 s
Wall time: 56.3 s


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62513863 entries, 0 to 62513862
Data columns (total 7 columns):
 #   Column         Dtype  
---  ------         -----  
 0   time           object 
 1   lat_min        float64
 2   lat_max        float64
 3   lon_min        float64
 4   lon_max        float64
 5   rain (mm/day)  float64
 6   model          object 
dtypes: float64(5), object(2)
memory usage: 3.3+ GB


| Team Member   | Operating System | RAM | Processor | Is SSD | Time taken |
|:-------------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Vera Cui      | macOS            | 16GB| M1        | No     | 56.3 s     |
| Lynn Wu       |                  |     |           |        |            |
| Jasmine Ortega|  MacOS           |8GB  |    M1     |  yes   |1 min 9s    |
| Yike Shi      |                  |     |           |        |            |

As is, the csv file took 1 minute and 46 seconds to load. From `.info()` we can see that the df consists of 6 columns all of the dtype `float64`. To reduce memory usage, we will first convert the data type to `float32` and `float16`, both which will reduce memory used, as shown below.

In [12]:
print(f"Memory usage with float64: {df.memory_usage().sum() / 1e6:.2f} MB")
print(f"Memory usage with float32: {df.astype('float32', errors='ignore').memory_usage().sum() / 1e6:.2f} MB")
print(f"Memory usage with float16: {df.astype('float16', errors='ignore').memory_usage().sum() / 1e6:.2f} MB")

Memory usage with float64: 3500.78 MB
Memory usage with float32: 2250.50 MB
Memory usage with float16: 1625.36 MB


#### Approaches to reduce memory usage while performing the EDA: *changing datatype*
##### Convert to float32

In [13]:
%%time
dtypes = {"lat_min" : "float32",
         "lat_max" : "float32",
         "lon_min" : "float32",
         "lon_max" : "float32",
         "rain (mm/day)" : "float32",
          "model" : "string"
        }

df_float32 = pd.read_csv('../data/processed/combined_data.csv', dtype=dtypes)

CPU times: user 42.4 s, sys: 5.5 s, total: 47.9 s
Wall time: 51.1 s


In [14]:
df_float32.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62513863 entries, 0 to 62513862
Data columns (total 7 columns):
 #   Column         Dtype  
---  ------         -----  
 0   time           object 
 1   lat_min        float32
 2   lat_max        float32
 3   lon_min        float32
 4   lon_max        float32
 5   rain (mm/day)  float32
 6   model          string 
dtypes: float32(5), object(1), string(1)
memory usage: 2.1+ GB


##### Convert to float16

In [15]:
%%time
dtypes = {"lat_min" : "float16",
         "lat_max" : "float16",
         "lon_min" : "float16",
         "lon_max" : "float16",
         "rain (mm/day)" : "float16",
        "model" : "string"
        }

df_float16 = pd.read_csv('../data/processed/combined_data.csv', dtype=dtypes)

CPU times: user 43.6 s, sys: 4.56 s, total: 48.2 s
Wall time: 50.5 s


In [16]:
df_float16 

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-35.43750,-33.56250,141.500,143.500,0.000000,MPI-ESM-1-2-HAM
1,1889-01-02 12:00:00,-35.43750,-33.56250,141.500,143.500,0.000000,MPI-ESM-1-2-HAM
2,1889-01-03 12:00:00,-35.43750,-33.56250,141.500,143.500,0.000000,MPI-ESM-1-2-HAM
3,1889-01-04 12:00:00,-35.43750,-33.56250,141.500,143.500,0.000000,MPI-ESM-1-2-HAM
4,1889-01-05 12:00:00,-35.43750,-33.56250,141.500,143.500,0.000000,MPI-ESM-1-2-HAM
...,...,...,...,...,...,...,...
62513858,2014-12-27 12:00:00,-30.15625,-29.21875,153.125,154.375,6.691406,SAM0-UNICON
62513859,2014-12-28 12:00:00,-30.15625,-29.21875,153.125,154.375,7.863281,SAM0-UNICON
62513860,2014-12-29 12:00:00,-30.15625,-29.21875,153.125,154.375,10.007812,SAM0-UNICON
62513861,2014-12-30 12:00:00,-30.15625,-29.21875,153.125,154.375,8.539062,SAM0-UNICON


As demonstrated, changing `float64` to less precise datatypes reduced runtimes. Interestingly, it looks like `float16` (50s) took almost as long to load as the more precise `float32` (58s). 

#### Approaches to reduce memory usage while performing the EDA: *Loading in chunks*

In [17]:
%%time

chunk = pd.read_csv("../data/processed/combined_data.csv", chunksize=10_000_000, iterator=True)
df = pd.concat(chunk)
df

CPU times: user 45.1 s, sys: 8.38 s, total: 53.5 s
Wall time: 57.2 s


,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.244226e-13,MPI-ESM-1-2-HAM
1,1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.217326e-13,MPI-ESM-1-2-HAM
2,1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.498125e-13,MPI-ESM-1-2-HAM
3,1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.251282e-13,MPI-ESM-1-2-HAM
4,1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.270161e-13,MPI-ESM-1-2-HAM
...,...,...,...,...,...,...,...
62513858,2014-12-27 12:00:00,-30.157068,-29.214660,153.1250,154.3750,6.689683e+00,SAM0-UNICON
62513859,2014-12-28 12:00:00,-30.157068,-29.214660,153.1250,154.3750,7.862555e+00,SAM0-UNICON
62513860,2014-12-29 12:00:00,-30.157068,-29.214660,153.1250,154.3750,1.000503e+01,SAM0-UNICON
62513861,2014-12-30 12:00:00,-30.157068,-29.214660,153.1250,154.3750,8.541592e+00,SAM0-UNICON


Loading the data in chunks of 10,000,000 reduced the loading time to 1min 7s. Let's combine the `float32` strategy with loading in chunks! (Note: we use `float32` instead of `float16` because half precision types are not supported when loading the data in chunks.  

In [2]:
%%time

dtypes = {"lat_min" : "float32",
         "lat_max" : "float32",
         "lon_min" : "float32",
         "lon_max" : "float32",
         "rain (mm/day)" : "float32",
         "model" : "string"
        }

final_df = pd.DataFrame()

chunk = pd.read_csv("../data/processed/combined_data.csv", chunksize=10_000_000, iterator=True, dtype=dtypes)
final_df = pd.concat(chunk)

NameError: name 'pd' is not defined

In [19]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62513863 entries, 0 to 62513862
Data columns (total 7 columns):
 #   Column         Dtype  
---  ------         -----  
 0   time           object 
 1   lat_min        float32
 2   lat_max        float32
 3   lon_min        float32
 4   lon_max        float32
 5   rain (mm/day)  float32
 6   model          string 
dtypes: float32(5), object(1), string(1)
memory usage: 2.1+ GB


We successfully reduced the load time from 1min 9s to 54.9s. 


**Optimized data loading:**

| Team Member   | Operating System | RAM | Processor | Is SSD | Time taken |
|:-------------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Vera Cui      | macOS            | 16GB| M1        | No     | 51.4 s     |
| Lynn Wu       |                  |     |           |        |            |
| Jasmine Ortega|  MacOS           |8GB  |    M1     |  yes   |54.9s       |
| Yike Shi      |                  |     |           |        |            |

Now, we will load the data (optimized) and conduct a simple EDA in which we will count the number of each model in the dataframe. 

In [4]:
%%time

dtypes = {"lat_min" : "float32",
         "lat_max" : "float32",
         "lon_min" : "float32",
         "lon_max" : "float32",
         "rain (mm/day)" : "float32",
         "model" : "string"
        }

final_df = pd.DataFrame()

chunk = pd.read_csv("../data/processed/combined_data.csv", chunksize=10_000_000, iterator=True, dtype=dtypes)
final_df = pd.concat(chunk)

# EDA
model_count = final_df["model"].value_counts()
model_count

CPU times: user 50.4 s, sys: 3.2 s, total: 53.6 s
Wall time: 54.7 s


MPI-ESM1-2-HR       5154240
NorESM2-MM          3541230
CMCC-CM2-SR5        3541230
TaiESM1             3541230
CMCC-ESM2           3541230
CMCC-CM2-HR4        3541230
SAM0-UNICON         3541153
GFDL-CM4            3219300
FGOALS-f3-L         3219300
GFDL-ESM4           3219300
MRI-ESM2-0          3037320
EC-Earth3-Veg-LR    3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM5-0           1609650
INM-CM4-8           1609650
KIOST-ESM           1287720
FGOALS-g3           1287720
AWI-ESM-1-1-LR       966420
MPI-ESM1-2-LR        966420
NESM3                966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
SYD.csv               46020
Name: model, dtype: Int64

**Python data load + EDA time:**

| Team Member   | Operating System | RAM | Processor | Is SSD | Time taken |
|:-------------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Vera Cui      | macOS            | 16GB| M1        | No     | 55.6 s     |
| Lynn Wu       |                  |     |           |        |            |
| Jasmine Ortega|  MacOS           |8GB  |    M1     |  yes   |58.5 s      |
| Yike Shi      |                  |     |           |        |            |

It looks like it takes ~1 minute to load and execute the EDA using the datatype `float32` and loading in chunks of 10,000,0000. 

(We attempted to plot the value counts, however, it consistently crashed the kernel on Mac M1) 

In [21]:
# alt.data_transformers.disable_max_rows()
# alt.renderers.enable('mimetype')

RendererRegistry.enable('mimetype')

In [ ]:
# this kills my kernel but you can try it lol 
# count_plot = alt.Chart(final_df).mark_bar().encode(x='model', y='count()')# this kills my kernel but you can try it lol 
# count_plot

#### Transform Python df to R Parquet 

We chose to use a parquet file to transfer the dataframe from Python to R because parquet files work well with rarrow. rarrow is ideal because by default, it reads and processes data in chunks, which we saw greatly improved our CSV loading time.

In [5]:
final_df.to_parquet("../data/processed/combined_data.parquet")

#### Perform a simple EDA in R

In [6]:
%load_ext rpy2.ipython

In [14]:
%%time 
%%R
library(dplyr)
library(arrow)


df <- open_dataset("../data/processed/combined_data.parquet") 

eda <- df |> count(model)
eda

result <- eda %>% collect
print(result, n=28)

# A tibble: 28 × 2
   model                  n
   <chr>              <int>
 1 MPI-ESM-1-2-HAM   966420
 2 AWI-ESM-1-1-LR    966420
 3 CMCC-CM2-HR4     3541230
 4 MRI-ESM2-0       3037320
 5 CMCC-ESM2        3541230
 6 NESM3             966420
 7 MPI-ESM1-2-HR    5154240
 8 CMCC-CM2-SR5     3541230
 9 MIROC6           2070900
10 SAM0-UNICON      3541153
11 NorESM2-LM        919800
12 EC-Earth3-Veg-LR 3037320
13 FGOALS-g3        1287720
14 INM-CM4-8        1609650
15 TaiESM1          3541230
16 GFDL-ESM4        3219300
17 ACCESS-CM2       1932840
18 GFDL-CM4         3219300
19 BCC-CSM2-MR      3035340
20 NorESM2-MM       3541230
21 SYD.csv            46020
22 KIOST-ESM        1287720
23 CanESM5           551880
24 FGOALS-f3-L      3219300
25 INM-CM5-0        1609650
26 ACCESS-ESM1-5    1610700
27 MPI-ESM1-2-LR     966420
28 BCC-ESM1          551880
CPU times: user 3.24 s, sys: 641 ms, total: 3.88 s
Wall time: 1.98 s


**R data load + EDA time:**

| Team Member   | Operating System | RAM | Processor | Is SSD | Time taken |
|:-------------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Vera Cui      | macOS            | 16GB| M1        | No     | 6.01 s     |
| Lynn Wu       |                  |     |           |        |            |
| Jasmine Ortega|  MacOS           |8GB  |    M1     |  yes   |2.49 s      |
| Yike Shi      |                  |     |           |        |            |

Overall, using a parquet file to transfer a Python dataframe to R was extremely effective. Considering the sheer volume of data we have (62 million rows), it's amazing that R was able to compute value counts in 2.5 seconds! Compare to Python, which took 58.5 seconds to complete the same task. From this simple experiment alone, it is clear to see the value of using an optimized storage file like a parquet over a CSV. 

In [19]:
%%R

library(ggplot2)
library(tidyverse)

count_eda <- result |>
  group_by(model) |>
  mutate(count = n())
            
count_eda

# A tibble: 28 × 3
# Groups:   model [28]
   model                 n count
   <chr>             <int> <int>
 1 MPI-ESM-1-2-HAM  966420     1
 2 AWI-ESM-1-1-LR   966420     1
 3 CMCC-CM2-HR4    3541230     1
 4 MRI-ESM2-0      3037320     1
 5 CMCC-ESM2       3541230     1
 6 NESM3            966420     1
 7 MPI-ESM1-2-HR   5154240     1
 8 CMCC-CM2-SR5    3541230     1
 9 MIROC6          2070900     1
10 SAM0-UNICON     3541153     1
# … with 18 more rows
